In [2]:
import sqlite3
from sqlite3 import*
import pandas as pd
import sys
import os


In [12]:
os.chdir('\Users\santo\Downloads\1602668798_SQLFiles_Tier_2_updated_10_13_2020 (4)\SQLFiles Tier 2')
database='\Users\santo\Downloads\1602668798_SQLFiles_Tier_2_updated_10_13_2020 (4)\SQLFiles Tier 2\sqlite_db_pythonsqlite.db'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \UXXXXXXXX escape (<ipython-input-12-2cd16317a0ba>, line 1)

In [ ]:
conn=sqlite3.connect(database)
cur=conn.cursor()

In [ ]:
query0="""
        SELECT name FROM sqlite_master WHERE type='table' ;
        """
cur.execute(query0)
rows =cur.fetchall()
cur.fetchone()
print(rows)
print(" \n")

In [ ]:

query01="""
        SELECT name FROM PRAGMA_TABLE_INFO('Members');
        """
cur.execute(query01)
rows1 =cur.fetchall()
print(rows1)
print(" \n")
query02="""
        SELECT name FROM PRAGMA_TABLE_INFO('Bookings');
        """
cur.execute(query02)
rows1 =cur.fetchall()
print(rows1)
print(" \n")
query03="""
        SELECT name FROM PRAGMA_TABLE_INFO('Facilities');
        """
cur.execute(query03)
rows1 =cur.fetchall()
print(rows1)
print(" \n")


In [ ]:
#Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [ ]:
query1="""
SELECT name FROM `Facilities` where membercost>0 ;   
"""
cur.execute(query1)
rows=cur.fetchall()
print("name")
for row in rows:
    print(row)

In [ ]:
#Q2: How many facilities do not charge a fee to members?

In [11]:
query2="""
SELECT count(name) FROM `Facilities` where membercost=0 ;   
"""
cur.execute(query2)
rows=cur.fetchall()
print("count")
for row in rows:
    print(row)

NameError: name 'cur' is not defined

In [ ]:
#Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
query3="""
SELECT facid, name, membercost, monthlymaintenance FROM `Facilities` where membercost>0 
and membercost<0.2*monthlymaintenance ; 
"""
cur.execute(query3)
rows=cur.fetchall()
print("facid, name, membercost, monthlymaintenance")
for row in rows:
    print(row)

In [ ]:
#Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [ ]:
query4="""
SELECT * FROM `Facilities` where facid in (1,5);
"""
cur.execute(query4)
rows=cur.fetchall()
print("bookid, facid, memid, starttime, slots")
for row in rows:
    print(row)

In [ ]:
#Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [ ]:
query5="""
SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 THEN 'expensive' 
ELSE 'cheap' END as cost FROM `Facilities` ;
"""
cur.execute(query5)
rows=cur.fetchall()
print("name, monthlymaintenance, cost")
for row in rows:
    print(row)

In [ ]:
#Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [ ]:
query6="""
SELECT  firstname, surname  FROM `Members` where joindate=(SELECT max(joindate) FROM `Members`) ;
"""
cur.execute(query6)
rows=cur.fetchall()
print("firstname, surname")
for row in rows:
    print(row)

In [ ]:
#Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [ ]:
query7="""
SELECT  distinct firstname ||' ' || surname  as membername, name as facility From `Members`  
join `Bookings` on Members.memid=Bookings.memid join `Facilities` 
on Bookings.facid=Facilities.facid where name like 'Tennis%' order by membername, facility; 
"""
cur.execute(query7)
rows=cur.fetchall()
print("membername,facility")
for row in rows:
    print(row)

In [ ]:
#Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [ ]:
query8="""
SELECT  name as facility, firstname ||' ' || surname as membername,   
CASE WHEN Members.memid=0 THEN guestcost*slots*2 ELSE membercost*slots*2 END 
as cost  From `Members`  join `Bookings` on Members.memid=Bookings.memid 
join `Facilities` on Bookings.facid=Facilities.facid where starttime 
like '2012-09-14%' and ((Members.memid!=0 and membercost*slots*2 >30) 
or (Members.memid=0 and guestcost*slots*2 >30)) order by cost desc;
"""
cur.execute(query8)
rows=cur.fetchall()
print("facility, membername, cost")
for row in rows:
    print(row)

In [ ]:
#Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
query9="""
SELECT  facility, membername, cost from (
SELECT  name as facility, firstname ||' ' || surname as membername, Members.memid, slots, membercost, guestcost,  
CASE WHEN Members.memid=0 THEN guestcost*slots*2 
ELSE membercost*slots*2 END as cost  From `Members`  join `Bookings` 
on Members.memid=Bookings.memid 
join `Facilities` on Bookings.facid=Facilities.facid 
where starttime like '2012-09-14%' ) as bookings_info 
where cost > 30 order by cost desc; 
"""
cur.execute(query9)
rows=cur.fetchall()
print("facility, membername, cost")
for row in rows:
    print(row)

In [ ]:
#Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [ ]:
query10="""
SELECT  name as facility, sum(CASE WHEN Bookings.memid=0 THEN guestcost*slots*2 
ELSE membercost*slots*2 END) as revenue From `Bookings`  join `Facilities` 
on Bookings.facid=Facilities.facid group by facility having revenue <1000 order by revenue;
"""
cur.execute(query10)
rows=cur.fetchall()
print("facility, revenue")
for row in rows:
    print(row)

In [ ]:
#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [ ]:
query11="""
         SELECT distinct firstname ||' ' ||  surname as member, (select CASE WHEN b.memid=0 THEN ' ' ELSE firstname ||' ' ||  surname END as recommender FROM `Members` b where a.recommendedby=b.memid ) as recommender from `Members` a order by member ;
        """
cur.execute(query11)
rows=cur.fetchall()
print("member, recommender")
for row in rows:
    print(row)

In [ ]:
#Q12: Find the facilities with their usage by member, but not guests

In [ ]:
query12="""
        SELECT  a.facility, count(a.membername) as No_of_members from 
        (SELECT  distinct name as facility, firstname ||' ' ||  surname as membername  From `Members`  
        join `Bookings` on Members.memid=Bookings.memid join `Facilities` 
        on Bookings.facid=Facilities.facid 
        where Members.memid!=0 order by membername, facility ) as a group by facility; 
        """
cur.execute(query12)
rows = cur.fetchall()
print("facility,No_of_members")
for row in rows:
    print(row)  

In [ ]:
#Q13: Find the facilities usage by month, but not guests

In [ ]:
query13="""
         select name as facility, a.month, count(name) as facility_usage from `Facilities` join (SELECT facid, strftime('%m',starttime) as month, starttime  from `Bookings` where Bookings.memid!=0) as a on a.facid=Facilities.facid group by a.month, facility ;
        """
cur.execute(query13)
rows = cur.fetchall()
print("facility,month,facility_usage")
for row in rows:
    print(row)